In [29]:
pip install blueprints.exploration 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement blueprints.exploration (from versions: none)
ERROR: No matching distribution found for blueprints.exploration


In [36]:
pip install textacy

  Using cached textacy-0.13.0-py3-none-any.whl (210 kB)
     ---------------------------------------- 12.3/12.3 MB 6.5 MB/s eta 0:00:00
  Using cached floret-0.10.3-cp39-cp39-win_amd64.whl (233 kB)
  Using cached pyphen-0.14.0-py3-none-any.whl (2.0 MB)
  Using cached spacy_loggers-1.0.4-py3-none-any.whl (11 kB)
  Using cached thinc-8.1.10-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached murmurhash-1.0.9-cp39-cp39-win_amd64.whl (18 kB)
     ------------------------------------- 483.6/483.6 kB 10.1 MB/s eta 0:00:00
  Using cached cymem-2.0.7-cp39-cp39-win_amd64.whl (30 kB)
  Using cached pathy-0.10.2-py3-none-any.whl (48 kB)
  Using cached preshed-3.0.8-cp39-cp39-win_amd64.whl (96 kB)
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached wasabi-1.1.2-py3-none-any.whl (27 kB)
     ---------------------------------------- 7.4/7.4 MB 8.3 MB/s eta 0:00:00
Note:

In [50]:
import tensorflow as tf
import pandas as pd
import sqlite3
import re
from collections import Counter
from tqdm import tqdm
tqdm.pandas()
import html
import textacy
import textacy.preprocessing as tprep
from textacy.preprocessing.resources import RE_URL


### GPU 설정

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' : 
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17064807794730865042
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2258003559
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8662762654857416553
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


#### 데이터 로드하기

In [12]:
post_df = pd.read_csv('./data/rspct_autos.tsv.gz', sep='\t')
subred_file = pd.read_csv('./data/subreddit_info.csv.gz').set_index(['subreddit'])

df = post_df.join(subred_file, on='subreddit')

In [13]:
df.head()

,id,subreddit,title,selftext,category_1,category_2,category_3,in_data,reason_for_exclusion
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,autos,harley davidson,NaN,True,NaN
1,5s0q8r,Mustang,Roush vs Shleby GT500,"I am trying to determine which is faster, and ...",autos,ford,NaN,True,NaN
2,5z3405,Volkswagen,2001 Golf Wagon looking for some insight,Hello! <lb><lb>Trying to find some information...,autos,VW,NaN,True,NaN
3,7df18v,Lexus,IS 250 Coolant Flush/Change,https://www.cars.com/articles/how-often-should...,autos,lexus,NaN,True,NaN
4,5tpve8,volt,Gen1 mpg w/ dead battery?,"Hi, new to this subreddit. I'm considering bu...",autos,chevrolet,NaN,True,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    20000 non-null  object
 1   subreddit             20000 non-null  object
 2   title                 20000 non-null  object
 3   selftext              20000 non-null  object
 4   category_1            20000 non-null  object
 5   category_2            20000 non-null  object
 6   category_3            0 non-null      object
 7   in_data               20000 non-null  bool  
 8   reason_for_exclusion  0 non-null      object
dtypes: bool(1), object(8)
memory usage: 1.2+ MB


In [15]:
df['in_data'].value_counts()

True    20000
Name: in_data, dtype: int64

#### 속성 이름 표준화

In [16]:
# 열 목록 확인하기
print(df.columns)

Index(['id', 'subreddit', 'title', 'selftext', 'category_1', 'category_2',
       'category_3', 'in_data', 'reason_for_exclusion'],
      dtype='object')


In [17]:
# 현재 열 이름 새 이름으로 매핑하기
# None으로 매핑된 열과 언급되지 않은 열은 삭제됨.
column_mapping = {
    'id': 'id',
    'subreddit': 'subreddit',
    'title': 'title',
    'selftext': 'text',
    'category_1': 'category',
    'category_2': 'subcategory',
    'category_3': None,           # 데이터가 없다.
    'in_data': None,              # 필요없다.
    'reason_for_exclusion': None  # 필요없다.
}

# 나머지 열들을 정의하기
columns = [c for c in column_mapping.keys() if column_mapping[c] != None]

# 열들을 선택하고 이름 바꾸기
df = df[columns].rename(columns=column_mapping)

In [18]:
df.head()

,id,subreddit,title,text,category,subcategory
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,autos,harley davidson
1,5s0q8r,Mustang,Roush vs Shleby GT500,"I am trying to determine which is faster, and ...",autos,ford
2,5z3405,Volkswagen,2001 Golf Wagon looking for some insight,Hello! <lb><lb>Trying to find some information...,autos,VW
3,7df18v,Lexus,IS 250 Coolant Flush/Change,https://www.cars.com/articles/how-often-should...,autos,lexus
4,5tpve8,volt,Gen1 mpg w/ dead battery?,"Hi, new to this subreddit. I'm considering bu...",autos,chevrolet


In [19]:
# 데이터를 자동차 범주로 제한하기
df = df[df['category'] == 'autos']

In [20]:
# pd.options.display.max_colwidth = None ###
pd.options.display.max_colwidth = 100 ###
df.sample(1, random_state=7).T

,14356
id,7jc2k4
subreddit,volt
title,Dashcam for 2017 volt
text,Hello.<lb>I'm looking into getting a dashcam. <lb>Does anyone have any recommendations? <lb><lb>...
category,autos
subcategory,chevrolet


#### 데이터프레임 저장 및 로드

In [21]:
df.to_pickle('reddit_dataframe.pkl')

In [22]:
db_name = 'reddit-selfposts.db'
con = sqlite3.connect(db_name)
df.to_sql('post', con, index=False, if_exists='replace')
con.close()

In [23]:
# 데이터 프레임 복원하기
con = sqlite3.connect(db_name)
df = pd.read_sql('select * from post', con)
con.close()

### 텍스트 데이터 정리

#### 정규 표현식으로 노이즈 식별

In [24]:
text = """
After viewing the [PINKIEPOOL Trailer](https://www.youtu.be/watch?v=ieHRoHUg)
it got me thinking about the best match ups.
<lb>Here's my take:<lb><lb>[](/sp)[](/ppseesyou) Deadpool<lb>[](/sp)[](/ajsly)
Captain America<lb>"""

In [25]:
print(text)


After viewing the [PINKIEPOOL Trailer](https://www.youtu.be/watch?v=ieHRoHUg)
it got me thinking about the best match ups.
<lb>Here's my take:<lb><lb>[](/sp)[](/ppseesyou) Deadpool<lb>[](/sp)[](/ajsly)
Captain America<lb>


In [26]:
RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\]')

def impurity(text, min_len = 10) : 
    '''텍스트에서 의심스러운 문자의 비율을 반환'''
    if text == None or len(text) < min_len : 
        return 0 
    else : 
        return len(RE_SUSPICIOUS.findall(text)) / len(text)
    
print(impurity(text))

0.09009009009009009


In [27]:
# 데이터프레임에 새 열 추가하기
df['impurity'] = df['text'].apply(impurity, min_len=10)

# 상위 3개 레코드 가져오기
df[['text', 'impurity']].sort_values(by='impurity', ascending=False).head(3)

,text,impurity
19682,Looking at buying a 335i with 39k miles and 11 months left on the CPO warranty. I asked the deal...,0.214716
12357,I'm looking to lease an a4 premium plus automatic with the nav package.<lb><lb>Vehicle Price:<ta...,0.165099
2730,Breakdown below:<lb><lb>Elantra GT<lb><lb>2.0L 4-cylinder<lb><lb>6-speed Manual Transmission<lb>...,0.139130


##### count_words 구현

In [28]:
def count_words(df, column='tokens', preprocess=None, min_freq=2) : 
    # 토큰 처리 및 counter 업데이트
    def update(doc) : 
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # counter생성 및 모든 데이터에서 적용
    counter = Counter()
    df[column].progress_map(update)

    # counter를 데이터프레임으로 전달(transform)
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    return freq_df.sort_values('freq', ascending=False)

In [29]:
# 정규 표현식으로 다른 태그가 있는지 확인하기
# <[\w/]*> : HTML과 유사한 구문 찾기(알파벳, 숫자, 밑줄 또는 /(슬래시)중 하나 찾기)
count_words(df, column='text', preprocess=lambda t: re.findall(r'<[\w/]*>', t))

100%|██████████| 20000/20000 [00:00<00:00, 193400.87it/s]


,freq
token,
<lb>,100729
<tab>,642


#### 정규 표현식으로 노이즈 제거

In [30]:
def clean(text) : 
    # &amp와 같은 html 이스케이프를 문자로 변환한다.
    text = html.unescape(text)
    # <tap>과 같은 태그를 공백으로 변환하기
    text = re.sub(r'<[^<>]*>', ' ', text)
    # [Some text](https://...)와 같은 마크다운 URL을 공백으로 변환한다.
    # r'\1 : 첫 번째 그룹에 일치하는 내용 반환
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # [0]과 같은 괄호 안의 텍스트 또는 코드를 공백으로 변환한다.
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # 특수 문자로만 구성된 문자열을 공백으로 변환한다. 
    # &#은 변환되지만 #cool은 변환되지 않음.
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # --- 또는 == 같은 하이픈으로 이뤄진 문자열을 공백으로 변환한다.
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:$|\s)', ' ', text)
    # 연속된 공백을 공백 하나로 변환한다.
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [31]:
# 샘플 텍스트에 clean함수 적용하고 결과 확인해보기
clean_text = clean(text)
print(clean_text)
print('Impurity:', impurity(clean_text))

After viewing the PINKIEPOOL Trailer it got me thinking about the best match ups. Here's my take: Deadpool Captain America
Impurity: 0.0


In [32]:
# 정리된 텍스트 불순물을 다시 확인하고 
# 필요하면 정리 단계 추가로 시행
df['clean_text'] = df['text'].map(clean)
df['impurity'] = df['clean_text'].apply(impurity, min_len=20)
df[['clean_text','impurity']].sort_values(by='impurity', ascending=False).head(3)

,clean_text,impurity
14058,"Mustang 2018, 2019, or 2020? Must Haves!! 1. Have a Credit score of 780\+ for the best low inter...",0.030864
18934,"At the dealership, they offered an option for foot-well illumination, but I cannot find any refe...",0.026455
16505,"I am looking at four Caymans, all are in a similar price range. The major differences are the mi...",0.024631


#### textacy를 사용한 문자 정규화

In [33]:
text = "The café “Saint-Raphaël” is loca-\nted on Côte dʼAzur."

In [45]:
# 다양한 하이픈과 따옴표를 표준화하고 악센트를 제거함
def normalize(text) : 
    text = tprep.normalize.hyphenated_words(text)
    text = tprep.normalize.quotation_marks(text)
    text = tprep.normalize.unicode(text)
    text = tprep.remove.accents(text)
    return text

In [46]:
print(normalize(text))

The cafe "Saint-Raphael" is located on Cote d'Azur.


#### textacy를 사용한 패턴 기반 데이터 마스킹

In [48]:
# 말뭉치에서 가장 자주 사용되는 url 찾기
count_words(df, column='clean_text', preprocess=RE_URL.findall).head(3)

100%|██████████| 20000/20000 [00:00<00:00, 32158.36it/s]


,freq
token,
www.getlowered.com,3
http://www.ecolamautomotive.com/#!2/kv7fq,2
https://www.reddit.com/r/Jeep/comments/4ux232/just_ordered_an_android_head_unit_joying_jeep/,2


In [51]:
if textacy.__version__ < '0.11':
    # as in book
    replace_urls = textacy.preprocessing.replace_urls
else:
    replace_urls = textacy.preprocessing.replace.urls

In [53]:
text = "Check out https://spacy.io/usage/spacy-101"

# 대체할 때 대체값으로 _URL_을 사용한다.
print(replace_urls(text))

Check out _URL_


In [54]:
# 정규화 및 데이터 마스킹 기능을 데이터에 적용하기
df['clean_text'] = df['clean_text'].map(replace_urls)
df['clean_text'] = df['clean_text'].map(normalize)

In [56]:
df.rename(columns={'text':'raw_text', 'clean_text':'text'}, inplace=True)
df.drop(columns=['impurity'], inplace=True)

In [57]:
# 데이터베이스에 저장
con = sqlite3.connect(db_name)
df.to_sql('post_cleaned', con, index=False, if_exists='replace')
con.close()

In [58]:
df.head()

,id,subreddit,title,raw_text,category,subcategory,text
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. This was before I knew anything about motorcycling ...,autos,harley davidson,Funny story. I went to college in Las Vegas. This was before I knew anything about motorcycling ...
1,5s0q8r,Mustang,Roush vs Shleby GT500,"I am trying to determine which is faster, and I've seen the dealership video with the two racing...",autos,ford,"I am trying to determine which is faster, and I've seen the dealership video with the two racing..."
2,5z3405,Volkswagen,2001 Golf Wagon looking for some insight,Hello! <lb><lb>Trying to find some information on replacing a 2001 Golf Wagon starter (gas).... ...,autos,VW,Hello! Trying to find some information on replacing a 2001 Golf Wagon starter (gas).... mine's g...
3,7df18v,Lexus,IS 250 Coolant Flush/Change,https://www.cars.com/articles/how-often-should-i-change-engine-coolant-1420680853669/<lb><lb>I h...,autos,lexus,_URL_ I have a IS 250 AWD from 2006. About 73K miles on it. I've never touched the engine radiat...
4,5tpve8,volt,Gen1 mpg w/ dead battery?,"Hi, new to this subreddit. I'm considering buying a Gen1 Volt, but I can't find any straight an...",autos,chevrolet,"Hi, new to this subreddit. I'm considering buying a Gen1 Volt, but I can't find any straight ans..."


### 토큰화

In [59]:
text = """
2019-08-10 23:32: @pete/@louis - I don't have a well-designed 
solution for today's problem. The code of module AC68 should be -1. 
Have to think a bit... #goodnight ;-) 😩😬"""

#### 정규 표현식을 사용한 토큰화

In [60]:
tokens = re.findall(r'\w\w+', text)
print(*tokens, sep='|')

2019|08|10|23|32|pete|louis|don|have|well|designed|solution|for|today|problem|The|code|of|module|AC68|should|be|Have|to|think|bit|goodnight
